In [1]:
import pandas as pd
import numpy as np

In [2]:
hopper_results = pd.read_csv("hopper_results_h100.csv")
print(f"Total configurations: {len(hopper_results)}")
print(f"Matrix sizes: {hopper_results[['M', 'N', 'K']].drop_duplicates().shape[0]}")
print(f"Unique Raster modes: {hopper_results['Raster'].unique()}")
print(f"Unique Swizzle values: {sorted(hopper_results['Swizzle'].unique())}")
print(f"Unique Decomposition modes: {hopper_results['Decomposition'].unique()}")
print(f"Unique Splits values: {sorted(hopper_results['Splits'].unique())}")

Total configurations: 672
Matrix sizes: 8
Unique Raster modes: ['Heuristic' 'Along N' 'Along M']
Unique Swizzle values: [np.int64(1), np.int64(2), np.int64(4), np.int64(8)]
Unique Decomposition modes: ['Heuristic' 'StreamK' 'SplitK' 'DataParallel']
Unique Splits values: [np.int64(1), np.int64(2), np.int64(3), np.int64(4)]


In [3]:
hopper_results.head(10)

,M,N,K,Raster,Swizzle,Decomposition,Splits,AvgRuntime_ms,TFLOPS,WorktileCount
0,128,128,128,Heuristic,1,Heuristic,1,0.006009,0.698008,1
1,128,128,128,Heuristic,2,Heuristic,1,0.005875,0.713900,1
2,128,128,128,Heuristic,4,Heuristic,1,0.006035,0.695010,1
3,128,128,128,Heuristic,8,Heuristic,1,0.005798,0.723435,1
4,128,128,128,Along N,1,Heuristic,1,0.006032,0.695379,1
5,128,128,128,Along N,2,Heuristic,1,0.005821,0.720492,1
6,128,128,128,Along N,4,Heuristic,1,0.005989,0.700321,1
7,128,128,128,Along N,8,Heuristic,1,0.005812,0.721603,1
8,128,128,128,Along M,1,Heuristic,1,0.005802,0.722956,1
9,128,128,128,Along M,2,Heuristic,1,0.006020,0.696710,1


In [4]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_white"

In [6]:
# Filter for square matrices where M=N=K
square_matrices = hopper_results[hopper_results["M"] == hopper_results["N"]]
square_matrices = square_matrices[square_matrices["N"] == square_matrices["K"]].copy()

# Create combined configuration labels
square_matrices["Raster_Swizzle"] = (
    square_matrices["Raster"]
    + " (Swizzle="
    + square_matrices["Swizzle"].astype(str)
    + ")"
)
square_matrices["Decomp_Splits"] = (
    square_matrices["Decomposition"]
    + " (Splits="
    + square_matrices["Splits"].astype(str)
    + ")"
)

print(f"Square matrix sizes (M=N=K): {sorted(square_matrices['M'].unique())}")

Square matrix sizes (M=N=K): [np.int64(128), np.int64(256), np.int64(512), np.int64(1024), np.int64(2048), np.int64(4096), np.int64(6144), np.int64(8192)]


In [7]:
# Raster + Swizzle Analysis
fig1 = make_subplots(
    rows=2,
    cols=1,
    row_heights=[0.65, 0.35],
    specs=[[{"type": "scatter"}], [{"type": "table"}]],
    subplot_titles=(
        "Performance vs Matrix Size: Raster Mode + Swizzle",
        "Best Configuration at Each Size",
    ),
    vertical_spacing=0.1,
)

# Add scatter points for all configurations
for config in square_matrices["Raster_Swizzle"].unique():
    data = square_matrices[square_matrices["Raster_Swizzle"] == config]
    fig1.add_trace(
        go.Scatter(
            x=data["M"],
            y=data["TFLOPS"],
            mode="markers",
            name=config,
            marker=dict(size=8, opacity=0.6),
        ),
        row=1,
        col=1,
    )

# Add line showing max performance at each size
max_perf = square_matrices.groupby("M")["TFLOPS"].max().reset_index()
fig1.add_trace(
    go.Scatter(
        x=max_perf["M"],
        y=max_perf["TFLOPS"],
        mode="lines+markers",
        name="Max Performance",
        line=dict(color="black", width=3, dash="dash"),
        marker=dict(size=10, color="black"),
    ),
    row=1,
    col=1,
)

# Find best config at each size
best_configs = []
for size in sorted(square_matrices["M"].unique()):
    subset = square_matrices[square_matrices["M"] == size]
    best = subset.loc[subset["TFLOPS"].idxmax()]
    best_configs.append(
        {
            "Size": f"{size}x{size}x{size}",
            "TFLOPS": f'{best["TFLOPS"]:.2f}',
            "Raster": best["Raster"],
            "Swizzle": str(best["Swizzle"]),
        }
    )

best_df = pd.DataFrame(best_configs)

# Add table with fixed height
num_rows = len(best_df)
cell_height = 25
header_height = 28

fig1.add_trace(
    go.Table(
        header=dict(
            values=[
                "<b>Size</b>",
                "<b>TFLOPS</b>",
                "<b>Raster Mode</b>",
                "<b>Swizzle</b>",
            ],
            fill_color="paleturquoise",
            align="center",
            font=dict(size=11),
            height=header_height,
        ),
        cells=dict(
            values=[
                best_df["Size"],
                best_df["TFLOPS"],
                best_df["Raster"],
                best_df["Swizzle"],
            ],
            fill_color="lavender",
            align="center",
            font=dict(size=10),
            height=cell_height,
        ),
    ),
    row=2,
    col=1,
)

fig1.update_xaxes(title_text="Matrix Size (M=N=K)", row=1, col=1)
fig1.update_yaxes(title_text="Performance (TFLOPS)", row=1, col=1)

# Calculate height to fit all rows without scrolling
total_height = 600 + (num_rows * cell_height) + header_height + 100
fig1.update_layout(height=total_height, showlegend=True, legend=dict(x=1.02, y=1))
fig1.show()

In [8]:
# Decomposition + Splits Analysis
fig2 = make_subplots(
    rows=2,
    cols=1,
    row_heights=[0.65, 0.35],
    specs=[[{"type": "scatter"}], [{"type": "table"}]],
    subplot_titles=(
        "Performance vs Matrix Size: Decomposition Mode + Splits",
        "Best Configuration at Each Size",
    ),
    vertical_spacing=0.1,
)

# Add scatter points for all configurations
for config in square_matrices["Decomp_Splits"].unique():
    data = square_matrices[square_matrices["Decomp_Splits"] == config]
    fig2.add_trace(
        go.Scatter(
            x=data["M"],
            y=data["TFLOPS"],
            mode="markers",
            name=config,
            marker=dict(size=8, opacity=0.6),
        ),
        row=1,
        col=1,
    )

# Add line showing max performance at each size
max_perf = square_matrices.groupby("M")["TFLOPS"].max().reset_index()
fig2.add_trace(
    go.Scatter(
        x=max_perf["M"],
        y=max_perf["TFLOPS"],
        mode="lines+markers",
        name="Max Performance",
        line=dict(color="black", width=3, dash="dash"),
        marker=dict(size=10, color="black"),
    ),
    row=1,
    col=1,
)

# Find best config at each size
best_configs = []
for size in sorted(square_matrices["M"].unique()):
    subset = square_matrices[square_matrices["M"] == size]
    best = subset.loc[subset["TFLOPS"].idxmax()]
    best_configs.append(
        {
            "Size": f"{size}x{size}x{size}",
            "TFLOPS": f'{best["TFLOPS"]:.2f}',
            "Decomposition": best["Decomposition"],
            "Splits": str(best["Splits"]),
        }
    )

best_df = pd.DataFrame(best_configs)

# Add table with fixed height
num_rows = len(best_df)
cell_height = 25
header_height = 28

fig2.add_trace(
    go.Table(
        header=dict(
            values=[
                "<b>Size</b>",
                "<b>TFLOPS</b>",
                "<b>Decomposition Mode</b>",
                "<b>Splits</b>",
            ],
            fill_color="paleturquoise",
            align="center",
            font=dict(size=11),
            height=header_height,
        ),
        cells=dict(
            values=[
                best_df["Size"],
                best_df["TFLOPS"],
                best_df["Decomposition"],
                best_df["Splits"],
            ],
            fill_color="lavender",
            align="center",
            font=dict(size=10),
            height=cell_height,
        ),
    ),
    row=2,
    col=1,
)

fig2.update_xaxes(title_text="Matrix Size (M=N=K)", row=1, col=1)
fig2.update_yaxes(title_text="Performance (TFLOPS)", row=1, col=1)

# Calculate height to fit all rows without scrolling
total_height = 600 + (num_rows * cell_height) + header_height + 100
fig2.update_layout(height=total_height, showlegend=True, legend=dict(x=1.02, y=1))
fig2.show()

In [9]:
# Overall Best Configuration Analysis (All 4 Parameters)
fig3 = make_subplots(
    rows=2,
    cols=1,
    row_heights=[0.65, 0.35],
    specs=[[{"type": "scatter"}], [{"type": "table"}]],
    subplot_titles=(
        "Overall Performance vs Matrix Size (All Configurations)",
        "Best Overall Configuration at Each Size",
    ),
    vertical_spacing=0.1,
)

# Add scatter points for all configurations (semi-transparent)
fig3.add_trace(
    go.Scatter(
        x=square_matrices["M"],
        y=square_matrices["TFLOPS"],
        mode="markers",
        name="All Configs",
        marker=dict(size=6, opacity=0.3, color="lightgray"),
    ),
    row=1,
    col=1,
)

# Add line showing max performance at each size
max_perf = square_matrices.groupby("M")["TFLOPS"].max().reset_index()
fig3.add_trace(
    go.Scatter(
        x=max_perf["M"],
        y=max_perf["TFLOPS"],
        mode="lines+markers",
        name="Max Performance",
        line=dict(color="red", width=4),
        marker=dict(size=12, color="red"),
    ),
    row=1,
    col=1,
)

# Find best overall config at each size (all 4 parameters)
best_configs = []
for size in sorted(square_matrices["M"].unique()):
    subset = square_matrices[square_matrices["M"] == size]
    best = subset.loc[subset["TFLOPS"].idxmax()]
    best_configs.append(
        {
            "Size": f"{size}x{size}x{size}",
            "TFLOPS": f'{best["TFLOPS"]:.2f}',
            "Raster": best["Raster"],
            "Swizzle": str(best["Swizzle"]),
            "Decomposition": best["Decomposition"],
            "Splits": str(best["Splits"]),
        }
    )

best_df = pd.DataFrame(best_configs)

# Add table with all 4 parameters
num_rows = len(best_df)
cell_height = 25
header_height = 28

fig3.add_trace(
    go.Table(
        header=dict(
            values=[
                "<b>Size</b>",
                "<b>TFLOPS</b>",
                "<b>Raster</b>",
                "<b>Swizzle</b>",
                "<b>Decomposition</b>",
                "<b>Splits</b>",
            ],
            fill_color="lightcoral",
            align="center",
            font=dict(size=11),
            height=header_height,
        ),
        cells=dict(
            values=[
                best_df["Size"],
                best_df["TFLOPS"],
                best_df["Raster"],
                best_df["Swizzle"],
                best_df["Decomposition"],
                best_df["Splits"],
            ],
            fill_color="mistyrose",
            align="center",
            font=dict(size=10),
            height=cell_height,
        ),
    ),
    row=2,
    col=1,
)

fig3.update_xaxes(title_text="Matrix Size (M=N=K)", row=1, col=1)
fig3.update_yaxes(title_text="Performance (TFLOPS)", row=1, col=1)

# Calculate height to fit all rows without scrolling
total_height = 600 + (num_rows * cell_height) + header_height + 100
fig3.update_layout(height=total_height, showlegend=True, legend=dict(x=1.02, y=1))
fig3.show()